In [1]:
from plot import *

In [2]:
model2bin_weekly_dose2data = get_model2bin_weekly_dose2data("data/buckets/")

In [22]:
model2bin_weekly_dose2measure2mean_ci = {}

ix = 2300
for model, bin_weekly_dose2data in model2bin_weekly_dose2data.items():
    print("model: " + str(model))
    bin_weekly_dose_vals = np.sort(np.array([int(k) for k in bin_weekly_dose2data.keys()]))
    mean_array = np.array()
    ci_array = np.array()
    for bin_weekly_dose_val in bin_weekly_dose_vals:
        data = bin_weekly_dose2data[bin_weekly_dose_val]
        all_a_star_a_hat, all_frac_incorrect, all_frac_correct, all_regret_expected, all_regret_observed = data
        
        end_frac_incorrect = all_frac_incorrect[:, ix]
        end_regret_expected = all_regret_expected[:, ix]
        end_regret_observed = all_regret_observed[:, ix]
    
        mean_array = np.append(mean_array, np.mean(end_frac_incorrect))

model: UCBDNet
[1 2 3 4 5]


AssertionError: 